# TruLens for LLMs: Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

Note: If you haven't already, make sure to set up your local .env file with your OpenAI and Huggingface Keys. Your .env file should be in the same directory that you run this notebook. If you need help, take a look at the [.env.example](https://github.com/truera/trulens_private/blob/e8b11c4689e644687d2eafe09d90d8d7774b581c/trulens_eval/trulens_eval/.env.example#L4)

## Import from LangChain and TruLens

In [1]:
from IPython.display import JSON

# imports from trulens to log and get feedback on chain
from trulens_eval.keys import *
from trulens_eval import TruChain, Feedback, OpenAI, Huggingface, db, tru

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


## Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [2]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)

## Send your first request to your new app, asking how to adopt a dog in spanish

In [3]:
prompt_input = 'Como adopto un perro?'
gpt3_response = chain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': '¡Adoptar un perro es una gran responsabilidad y una decisión importante! Si estás interesado en adoptar un perro, lo primero que debes hacer es investigar los refugios y organizaciones de rescate de animales en tu área. Estos lugares suelen tener perros de diferentes edades, tamaños y personalidades, por lo que es posible que puedas encontrar el compañero perfecto para ti.\n\nAntes de adoptar, asegúrate de tener en cuenta tus necesidades y estilo de vida para encontrar un perro que se adapte bien a tu hogar. También debes estar preparado para proporcionarle a tu nuevo compañero todo lo que necesita, incluyendo alimentos de calidad, atención médica regular y suficiente ejercicio y estimulación mental.\n\nCuando encuentres un perro que te guste, tendrás que completar un proceso de solicitud y evaluación de adopción. Este proceso puede variar dependiendo del refugio u organización, pero por lo general incluye una entrevista, una verificación d

# Instrument chain for logging with TruLens

In [4]:
truchain: TruChain = TruChain(chain, chain_id='Chain1_ChatApplication')

In [5]:
# Instrumented chain can operate like the original:

gpt3_response = truchain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': 'Adoptar un perro es un proceso emocionante y gratificante que puede tomar algo de tiempo y preparación. Aquí hay algunos pasos que puedes seguir para adoptar un perro:\n\n1. Investiga las diferentes organizaciones de rescate y refugios de animales en tu área. Estos lugares pueden tener una variedad de perros disponibles para la adopción, desde cachorros hasta perros mayores.\n\n2. Considera qué tipo de perro sería adecuado para ti y para tu estilo de vida. ¿Quieres un perro activo o uno que sea más tranquilo? ¿Tienes espacio suficiente en tu hogar para un perro grande o prefieres uno más pequeño?\n\n3. Llena la solicitud de adopción proporcionada por la organización de rescate o refugio. Esta solicitud te permitirá proporcionar información sobre tu hogar, estilo de vida y experiencia con perros.\n\n4. Visita el refugio o la organización de rescate para conocer a los perros disponibles para la adopción. Pregunta acerca de sus necesidades de 

In [6]:
# But can also produce a log or "record" of the execution of the chain:

gpt3_response, record = truchain.call_with_record(prompt_input)

JSON(record)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


<IPython.core.display.JSON object>

In [7]:
# We can log the records but first we need to log the chain itself:

tru.add_chain(chain_json=truchain.json)

In [8]:
# Now the record:

record_id = tru.add_record(
    prompt=prompt_input, # prompt input
    response=gpt3_response['text'], # LLM response
    record_json=record # record is returned by the TruChain wrapper
)

In [9]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

*** Creating new Endpoint singleton instance for name = huggingface ***
*** Creating huggingface endpoint ***


huggingface api: 0requests [00:00, ?requests/s]

*** Creating new TP singleton instance for name = None ***


In [10]:
# This might take a moment if the public api needs to load the language model
# used in the feedback function:
feedback_result = f_lang_match.run_on_record(
    chain_json=truchain.json, record_json=record
)

In [11]:
JSON(feedback_result)

<IPython.core.display.JSON object>

In [12]:
# Alternatively, run a collection of feedback functions:

feedback_results = tru.run_feedback_functions(
    record_json=record,
    feedback_functions=[f_lang_match]
)

In [13]:
display(feedback_results)

[{'_success': True,
  'feedback_id': 'feedback_hash_26c84c76d907d808c36028e024af63c0',
  'record_id': 'record_hash_3d3ce2e1eab00fceebd54330d2944074',
  'language_match': 0.005192771084693959}]

In [14]:
# These can be logged:

for result in feedback_results:
    tru.add_feedback(result)

## Run the TruLens dashboard to explore the quality of your LLM chain

In [15]:
tru.run_dashboard() # open a local streamlit app to explore

## Automatic Logging

The above logging and feedback function evaluation steps can be done by TruChain.

In [16]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    db=db
)

# Note: providing `db: TruDB` causes the above constructor to log the wrapped chain in the database specified.

# Note: any `feedbacks` specified here will be evaluated and logged whenever the chain is used.

Inserting chain and feedback function definitions to db.


In [17]:
truchain("This will be automatically logged.")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: This will be automatically logged.

      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  
> Finished chain.
Running feedback functions now.


{'prompt': 'This will be automatically logged.',
 'text': 'When someone says "this will be automatically logged," it means that a system or software is tracking the activity or event. This is common in many industries where it is necessary to maintain a record of all actions and events for legal, regulatory, or internal purposes. For example, in cybersecurity, all network and system traffic is automatically logged to track any potential security breaches or attacks. Similarly, in healthcare, patient records are automatically logged to ensure compliance with regulations and to provide accurate medical records. Consequently, it is important to be aware that your actions are being recorded and monitored in certain settings.'}

## Out-of-band Feedback evaluation

In the above example, the feedback function evaluation is done in the same process as the chain evaluation. The alternative approach is the use the provided persistent evaluator started via `tru.start_deferred_feedback_evaluator`. Then specify the `feedback_mode` for `TruChain` as `deferred` to let the evaluator handle the feedback functions.

For demonstration purposes, we start the evaluator here but it can be started in another process.

In [18]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    db=db,
    feedback_mode="deferred"
)

Inserting chain and feedback function definitions to db.


In [19]:
tru.start_deferred_feedback_evaluator()

In [20]:
truchain("This will be logged by deferred evaluator.")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: This will be logged by deferred evaluator.

> Finished chain.
Inserting feedback for future evaluation feedback_hash_26c84c76d907d808c36028e024af63c0


{'prompt': 'This will be logged by deferred evaluator.',
 'text': 'Deferred evaluation is a programming technique that defers the evaluation of an expression until its value is actually needed. This technique helps to improve performance and optimize memory usage. \n\nWhen a statement is logged by deferred evaluator, it means that the evaluation of that statement will be postponed until a later time. This is useful in situations where the statement is time-consuming or resource-intensive, and executing it immediately would negatively impact the performance of the application. \n\nThe deferred evaluator will keep track of the statement and execute it at an appropriate time, ensuring that the application runs smoothly and efficiently. This technique is commonly used in functional programming languages and frameworks like Haskell and ReactiveX.'}

Starting run for row 2.


Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.